In [32]:
from vmap_utils import PEPS_Model
import quimb.tensor as qtn
import pickle
import torch
import numpy as np
import torch.nn as nn

Lx = 4
Ly = 2
D = 4
chi = 4*D
seed = 42
# pwd = '/home/sijingdu/TNVMC/VMC_code/vmc_torch/vmc_torch/experiment/vmap'
# skeleton = pickle.load(open(pwd+f'/{Lx}x{Ly}/heis/D={D}/peps_skeleton.pkl', 'rb'))
# params = pickle.load(open(pwd+f'/{Lx}x{Ly}/heis/D={D}/peps_su_params.pkl', 'rb'))
# peps = qtn.unpack(params, skeleton)
peps = qtn.PEPS.rand(Lx, Ly, D, seed=seed)

model = PEPS_Model(tn=peps, max_bond=chi)

B = 1024
random_binary_config = torch.randint(0, 2, (B, Lx*Ly)).int()

model(random_binary_config)

tensor([-1167.8648,  -290.4566,  -898.9068,  ...,  1102.4895,   491.9807,
         2942.9152], dtype=torch.float64, grad_fn=<MulBackward0>)

In [ ]:
from vmc_torch.hamiltonian_torch import spin_Heisenberg_square_lattice_torch

H = spin_Heisenberg_square_lattice_torch(Lx, Ly, J=1.0, total_sz=0)

np.linalg.eigvals(H.to_dense())[0]/(Lx*Ly)  # ground state energy per site check

np.float64(-0.5366333070821349)

: 

: 

In [203]:
import quimb as qu
class PEPS_Model_reuse(nn.Module):
    def __init__(self, tn, max_bond, dtype=torch.float64):
        import quimb as qu
        import quimb.tensor as qtn
        super().__init__()
        
        params, skeleton = qtn.pack(tn)
        self.dtype = dtype
        self.skeleton = skeleton
        self.bMPS_x_skeletons = {}
        self.bMPS_y_skeletons = {}
        self.bMPS_params_x_in_dims = None
        self.bMPS_params_y_in_dims = None
        self.chi = max_bond

        # for torch, further flatten pytree into a single list
        params_flat, params_pytree = qu.utils.tree_flatten(
            params, get_ref=True
        )
        self.params_pytree = params_pytree

        # register the flat list parameters
        self.params = torch.nn.ParameterList([
            torch.as_tensor(x, dtype=self.dtype) for x in params_flat
        ])
    
    def cache_bMPS_skeleton(self, x):
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        tn = qtn.unpack(params, self.skeleton)
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})
        env_x = amp.compute_x_environments(max_bond=self.chi, cutoff=0.0)
        bMPS_params_dict = {}
        for key, tn in env_x.items():
            bMPS_params, skeleton = qtn.pack(tn)
            env_x[key] = skeleton
            bMPS_params_dict[key] = bMPS_params

        self.bMPS_x_skeletons = env_x
        bMPS_params_x_in_dims = qu.utils.tree_map(lambda _: 0, bMPS_params_dict)
        self.bMPS_params_x_in_dims = bMPS_params_x_in_dims

        env_y = amp.compute_y_environments(max_bond=self.chi, cutoff=0.0)
        bMPS_params_dict = {}
        for key, tn in env_y.items():
            bMPS_params, skeleton = qtn.pack(tn)
            env_y[key] = skeleton
            bMPS_params_dict[key] = bMPS_params
        self.bMPS_y_skeletons = env_y
        bMPS_params_y_in_dims = qu.utils.tree_map(lambda _: 0, bMPS_params_dict)
        self.bMPS_params_y_in_dims = bMPS_params_y_in_dims

    
    def cache_bMPS_params_vmap(self, x):
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        def cache_bMPS_params_single(x_single, params):
            tn = qtn.unpack(params, self.skeleton)
            amp = tn.isel({tn.site_ind(site): x_single[i] for i, site in enumerate(tn.sites)})
            env_x = amp.compute_x_environments(max_bond=self.chi, cutoff=0.0)
            bMPS_params_x_dict = {}
            for key, btn in env_x.items():
                bMPS_params = btn.get_params()
                bMPS_params_x_dict[key] = bMPS_params
            bMPS_params_y_dict = {}
            env_y = amp.compute_y_environments(max_bond=self.chi, cutoff=0.0)
            for key, btn in env_y.items():
                bMPS_params = btn.get_params()
                bMPS_params_y_dict[key] = bMPS_params
            return bMPS_params_x_dict, bMPS_params_y_dict
        return torch.vmap(
            cache_bMPS_params_single,
            in_dims=(0, None),
        )(x, params)
    
    def amp_tn(self, x):
        params = qu.utils.tree_unflatten(self.params, self.params_pytree)
        tn = qtn.unpack(params, self.skeleton)
        # might need to specify the right site ordering here
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})
        return amp
    
    def amplitude(
        self,
        x,
        params,
        bMPS_key=None,
        bMPS_params_xmin=None,
        bMPS_params_xmax=None,
        bMPS_params_ymin=None,
        bMPS_params_ymax=None,
        selected_rows=None,
        selected_cols=None,
    ):
        tn = qtn.unpack(params, self.skeleton)
        # might need to specify the right site ordering here
        amp = tn.isel({tn.site_ind(site): x[i] for i, site in enumerate(tn.sites)})

        # replace the x-environment with the cached one
        if bMPS_params_xmin is not None and bMPS_params_xmax is not None and bMPS_key is not None:
            bMPS_min = qtn.unpack(bMPS_params_xmin, self.bMPS_x_skeletons[bMPS_key[0]])
            bMPS_max = qtn.unpack(bMPS_params_xmax, self.bMPS_x_skeletons[bMPS_key[1]])
            rows = amp.select([tn.row_tag(row) for row in selected_rows], which='any')
            amp_reuse = (bMPS_min|rows|bMPS_max)
            amp_reuse.view_as_(
                qtn.PEPS,
                site_tag_id = tn._site_tag_id,
                x_tag_id = tn._x_tag_id,
                y_tag_id = tn._y_tag_id,
                Lx = tn._Lx,
                Ly = tn._Ly,
                site_ind_id = tn._site_ind_id,
            )
            if self.chi > 0:
                amp_reuse.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[0, amp.Ly//2-1])
                amp_reuse.contract_boundary_from_ymax_(max_bond=self.chi, cutoff=0.0, yrange=[amp.Ly//2, amp.Ly-1])
            return amp_reuse.contract()
        # replace the y-environment with the cached one
        if bMPS_params_ymin is not None and bMPS_params_ymax is not None and bMPS_key is not None:
            bMPS_min = qtn.unpack(bMPS_params_ymin, self.bMPS_y_skeletons[bMPS_key[0]])
            bMPS_max = qtn.unpack(bMPS_params_ymax, self.bMPS_y_skeletons[bMPS_key[1]])
            cols = amp.select([tn.col_tag(col) for col in selected_cols], which='any')
            amp_reuse = (bMPS_min|cols|bMPS_max)
            amp_reuse.view_as_(
                qtn.PEPS,
                site_tag_id = tn._site_tag_id,
                x_tag_id = tn._x_tag_id,
                y_tag_id = tn._y_tag_id,
                Lx = tn._Lx,
                Ly = tn._Ly,
                site_ind_id = tn._site_ind_id,
            )
            if self.chi > 0:
                amp_reuse.contract_boundary_from_xmin_(max_bond=self.chi, cutoff=0.0, xrange=[0, amp.Lx//2-1])
                amp_reuse.contract_boundary_from_xmax_(max_bond=self.chi, cutoff=0.0, xrange=[amp.Lx//2, amp.Lx-1])
            return amp_reuse.contract()

        if self.chi > 0:
            amp.contract_boundary_from_ymin_(max_bond=self.chi, cutoff=0.0, yrange=[0, amp.Ly//2-1])
            amp.contract_boundary_from_ymax_(max_bond=self.chi, cutoff=0.0, yrange=[amp.Ly//2, amp.Ly-1])
        return amp.contract()
    
    def vamp(
        self,
        x,
        params,
        bMPS_key=None,
        bMPS_params_xmin=None,
        bMPS_params_xmax=None,
        bMPS_params_ymin=None,
        bMPS_params_ymax=None,
        selected_rows=None,
        selected_cols=None,
    ):
        params = qu.utils.tree_unflatten(params, self.params_pytree)
        if bMPS_params_xmin is not None and bMPS_params_xmax is not None:
            return torch.vmap(
                self.amplitude,
                in_dims=(
                    0,
                    None,
                    None,
                    self.bMPS_params_x_in_dims[bMPS_key[0]],
                    self.bMPS_params_x_in_dims[bMPS_key[1]],
                    None,
                    None,
                    None,
                    None,
                ),
            )(x, params, bMPS_key, bMPS_params_xmin, bMPS_params_xmax, bMPS_params_ymin, bMPS_params_ymax, selected_rows, selected_cols)
        
        if bMPS_params_ymin is not None and bMPS_params_ymax is not None:
            return torch.vmap(
                self.amplitude,
                in_dims=(
                    0,
                    None,
                    None,
                    None,
                    None,
                    self.bMPS_params_y_in_dims[bMPS_key[0]],
                    self.bMPS_params_y_in_dims[bMPS_key[1]],
                    None,
                    None,
                ),
            )(x, params, bMPS_key, bMPS_params_xmin, bMPS_params_xmax, bMPS_params_ymin, bMPS_params_ymax, selected_rows, selected_cols)

    def forward(
        self,
        x,
        bMPS_params_x_batched=None,
        bMPS_params_y_batched=None,
        selected_rows=None,
        selected_cols=None,
    ):
        bMPS_params_xmin = None
        bMPS_params_xmax = None
        bMPS_params_ymin = None
        bMPS_params_ymax = None

        if selected_rows is not None:
            bMPS_key = [('xmin', min(selected_rows)), ('xmax', max(selected_rows))]
            bMPS_params_xmin = bMPS_params_x_batched[bMPS_key[0]]
            bMPS_params_xmax = bMPS_params_x_batched[bMPS_key[1]]
        if selected_cols is not None:
            bMPS_key = [('ymin', min(selected_cols)), ('ymax', max(selected_cols))]
            bMPS_params_ymin = bMPS_params_y_batched[bMPS_key[0]]
            bMPS_params_ymax = bMPS_params_y_batched[bMPS_key[1]]
        
        return self.vamp(
            x,
            self.params,
            bMPS_key=bMPS_key,
            bMPS_params_xmin=bMPS_params_xmin,
            bMPS_params_xmax=bMPS_params_xmax,
            bMPS_params_ymin=bMPS_params_ymin,
            bMPS_params_ymax=bMPS_params_ymax,
            selected_rows=selected_rows,
            selected_cols=selected_cols,
        )

model_reuse = PEPS_Model_reuse(tn=peps, max_bond=chi)
model_reuse.cache_bMPS_skeleton(random_binary_config[0])
reuse_B = 10
amp_tns = [model_reuse.amp_tn(random_binary_config[i]) for i in range(reuse_B)]
B_bMPS_params_x_dict, B_bMPS_params_y_dict = model_reuse.cache_bMPS_params_vmap(random_binary_config[:reuse_B])
current_amps = model_reuse(random_binary_config[:reuse_B], bMPS_params_x_batched=B_bMPS_params_x_dict, bMPS_params_y_batched=B_bMPS_params_y_dict, selected_rows=(0,), selected_cols=None)
current_amps

tensor([-1167.8648,  -290.4566,  -898.9068,   462.7139,  -235.6403,  -131.4165,
        -1050.1250,   -21.7044,   -74.7150,   306.4095], dtype=torch.float64,
       grad_fn=<MulBackward0>)

In [204]:
[amp_tns[i].contract() for i in range(reuse_B)]

[tensor(-1167.8648, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-290.4566, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-898.9068, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(462.7139, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-235.6403, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-131.4165, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-1050.1250, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-21.7044, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(-74.7150, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(306.4095, dtype=torch.float64, grad_fn=<MulBackward0>)]

In [ ]:
import mpi4py.MPI as MPI
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()

@torch.inference_mode()
def evaluate_energy_reuse(fxs, v_model, H, current_amps, verbose=False):
    # Label each connected config with which sample it comes from to enable reuse

    B = fxs.shape[0]
    

    # cache skeleton for reuse
    v_model.cache_bMPS_skeleton(fxs[0]) 
    # cache bMPS params for reuse
    B_bMPS_params_x_dict, B_bMPS_params_y_dict = v_model.cache_bMPS_params_vmap(fxs)

    def detect_changed_row_col_pair(fx1, fx2):
        # currently only support nearest neighbor on square lattice
        Ly = v_model.skeleton._Ly
        changed_pos = torch.nonzero(fx1 - fx2)
        changed_pos_2d = []
        assert changed_pos.shape[0] <= 2, "Expect at most 2 on-site config changes"
        for pos in changed_pos:
            x, y = pos.item() // Ly, pos.item() % Ly
            changed_pos_2d.append( (x, y) )
        if len(changed_pos_2d) == 2:
            delta_row = abs(changed_pos_2d[0][0] - changed_pos_2d[1][0])
            delta_col = abs(changed_pos_2d[0][1] - changed_pos_2d[1][1])
            if delta_row <= delta_col:
                x1 = min(changed_pos_2d, key=lambda t: t[0])[0]
                row = True
                col = False
                return row, col, list(x for x in range(x1, x1+delta_row+1))
            else:
                y1 = min(changed_pos_2d, key=lambda t: t[1])[1]
                row = False
                col = True
                return row, col, list(y for y in range(y1, y1+delta_col+1))
        else:
            row = col = False
            return row, col, None
             
            
    # get connected configurations, coefficients and indices
    conn_eta_num = []
    conn_etas = []
    conn_eta_coeffs = []
    conn_eta_indices = []

    fx_ind = 0
    for fx in fxs:
        eta, coeffs = H.get_conn(fx)
        for i_eta in range(len(eta)):
            r, c, pos = detect_changed_row_col_pair(fx, eta[i_eta])
            conn_eta_indices.append( (fx_ind, i_eta, r, c, pos) )

        conn_eta_num.append(len(eta))
        conn_etas.append(torch.tensor(eta))
        conn_eta_coeffs.append(torch.tensor(coeffs))
        fx_ind += 1

    conn_etas = torch.cat(conn_etas, dim=0)
    conn_eta_coeffs = torch.cat(conn_eta_coeffs, dim=0)

    # group connected configs by changed row/col first
    # within row/col group, further group by position

    # select the indices where r==True and c==False
    # TODO: complete this part, form the pytree with batched leaves to input to reusbale PEPS amplitude calculation
    qu.utils.tree_map_list(lambda x: print(x), conn_eta_indices, is_leaf=lambda x: isinstance(x, tuple))
    # print(conn_eta_num)

    # if verbose:
    #     if RANK == 1:
    #         print(f'Prepared batched conn_etas and coeffs: {conn_etas.shape}, {conn_eta_coeffs.shape} (batch size {B})')

    # # calculate amplitudes for connected configs, in the future consider TN reuse to speed up calculation, TN reuse is controlled by a param that is not batched over (control flow?)
    # conn_amps = torch.cat([v_model(conn_etas[i:i+B]) for i in range(0, conn_etas.shape[0], B)])

    # # Local energy \sum_{s'} H_{s,s'} <s'|psi>/<s|psi>

    # local_energies = []
    # offset = 0
    # for b in range(B):
    #     n_conn = conn_eta_num[b]
    #     amps_ratio = conn_amps[offset:offset+n_conn] / current_amps[b]
    #     energy_b = torch.sum(conn_eta_coeffs[offset:offset+n_conn] * amps_ratio)
    #     local_energies.append(energy_b)
    #     offset += n_conn
    # local_energies = torch.stack(local_energies, dim=0)
    # if verbose:
    #     if RANK == 1:
    #         print(f'Batched local energies: {local_energies.shape}')

    # # Energy: (1/N) * \sum_s <s|H|psi>/<s|psi> = (1/N) * \sum_s \sum_{s'} H_{s,s'} <s'|psi>/<s|psi>
    # energy = torch.mean(local_energies)
    # if verbose:
    #     if RANK == 1:
    #         print(f'Energy: {energy.item()}')

    # return energy, local_energies

evaluate_energy_reuse(random_binary_config[:reuse_B], model_reuse, H, current_amps, verbose=True)

(0, 0, True, False, [0])
(0, 1, False, False, None)
(0, 2, False, True, [1])
(0, 3, False, True, [1])
(0, 4, True, False, [2])
(0, 5, False, True, [1])
(1, 0, True, False, [0])
(1, 1, False, False, None)
(1, 2, False, True, [0])
(1, 3, False, True, [1])
(1, 4, True, False, [1])
(1, 5, False, True, [0])
(1, 6, False, True, [1])
(1, 7, True, False, [2])
(1, 8, False, True, [0])
(1, 9, False, True, [1])
(1, 10, True, False, [3])
(2, 0, False, False, None)
(2, 1, False, True, [0])
(2, 2, True, False, [1])
(2, 3, False, True, [0])
(2, 4, False, True, [0])
(2, 5, False, True, [1])
(3, 0, True, False, [0])
(3, 1, False, False, None)
(3, 2, False, True, [1])
(3, 3, False, True, [0])
(3, 4, True, False, [2])
(3, 5, False, True, [1])
(4, 0, True, False, [0])
(4, 1, False, False, None)
(4, 2, False, True, [0])
(4, 3, False, True, [1])
(4, 4, True, False, [1])
(4, 5, False, True, [0])
(4, 6, False, True, [0])
(4, 7, True, False, [3])
(5, 0, False, False, None)
(5, 1, False, True, [1])
(5, 2, True,

/tmp/ipykernel_17041/2047412198.py:20: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  changed_pos = torch.nonzero(fx1 - fx2)


In [119]:
_EXTRA_PROPS = ('_site_tag_id', '_x_tag_id', '_y_tag_id', '_Lx', '_Ly', '_site_ind_id')
# get the extra properties of the tensor network
qtn_extra_props = {prop: getattr(peps, prop) for prop in _EXTRA_PROPS}
qtn_extra_props

{'_site_tag_id': 'I{},{}',
 '_x_tag_id': 'X{}',
 '_y_tag_id': 'Y{}',
 '_Lx': 4,
 '_Ly': 2,
 '_site_ind_id': 'k{},{}'}

In [88]:
model_reuse.bMPS_skeletons

{('xmax', 3): TensorNetwork(tensors=0, indices=0),
 ('xmax', 2): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4),
 ('xmax', 1): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4),
 ('xmax', 0): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4),
 ('xmin', 0): TensorNetwork(tensors=0, indices=0),
 ('xmin', 1): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4),
 ('xmin', 2): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4),
 ('xmin', 3): PEPS(tensors=2, indices=3, Lx=4, Ly=2, max_bond=4)}